## **Laboratorio 9**
## **Julio García Salas - 22076**
## **Sofía García - 22210**

# Modelos de Markov y Hidden Markov Models (HMM)

## 1. Diferencia entre Modelos de Markov y Hidden Markov Models

Un **Modelo de Markov (Markov Chain)** es un modelo probabilístico que describe una secuencia de posibles eventos en los que la probabilidad de cada evento depende únicamente del estado alcanzado en el evento anterior. Es decir, el proceso tiene la propiedad de **Markov**, también conocida como **memoria limitada**.

- En un Modelo de Markov clásico, los **estados son observables**.
- Se describe con una matriz de transición $A$ entre estados.

Un **Modelo Oculto de Markov (Hidden Markov Model, HMM)** es una extensión del modelo de Markov donde el **estado real no es observable directamente** (es "oculto"), pero se puede inferir a partir de **observaciones** que dependen probabilísticamente de los estados ocultos.

- En un HMM, se tiene:
  - Una secuencia de **estados ocultos** $q_1, q_2, ..., q_T$
  - Una secuencia de **observaciones** $o_1, o_2, ..., o_T$
  - Probabilidades de emisión $B$ que definen la probabilidad de observar $o_t$ dado el estado oculto $q_t$

## 2. ¿Qué son los factorial HMM?

Los **Factorial Hidden Markov Models (FHMM)** son una generalización de los HMM tradicionales. En lugar de tener una única cadena de estados ocultos, un FHMM utiliza **varias cadenas ocultas que evolucionan en paralelo** y que juntas determinan la distribución de las observaciones.

- Cada cadena de estados ocultos sigue un modelo de Markov independiente.
- Las observaciones se generan a partir de la **combinación de múltiples cadenas ocultas**.

Esto permite modelar **dependencias más complejas** entre los estados y las observaciones. Son útiles en contextos donde múltiples factores ocultos influyen en los datos observados (por ejemplo, reconocimiento de actividad humana, bioinformática, etc.).

## 3. Algoritmo Forward-Backward para HMM

El algoritmo **Forward-Backward** es una técnica de inferencia utilizada en HMMs para **calcular la probabilidad de una secuencia de observaciones** y para inferir la **probabilidad posterior de los estados ocultos** en cada tiempo dado.

Se divide en dos fases:

- **Forward ($\alpha$) paso**: calcula de manera recursiva la probabilidad de la secuencia observada hasta el tiempo $t$ y de estar en un estado $i$ en ese momento:
  
  $$
  \alpha_t(i) = P(o_1, o_2, ..., o_t, q_t = i \mid \lambda)
  $$

- **Backward ($\beta$) paso**: calcula la probabilidad de observar el resto de la secuencia desde $t+1$ en adelante, dado que el sistema está en el estado $i$ en el tiempo $t$:

  $$
  \beta_t(i) = P(o_{t+1}, o_{t+2}, ..., o_T \mid q_t = i, \lambda)
  $$

- Finalmente, la probabilidad posterior de estar en un estado $i$ en el tiempo $t$ dado la secuencia completa de observaciones es:

  $$
  \gamma_t(i) = \frac{\alpha_t(i) \cdot \beta_t(i)}{\sum_{j=1}^N \alpha_t(j) \cdot \beta_t(j)}
  $$

## 4. ¿Por qué es necesario el paso Backward?

El paso **Backward** permite incorporar la información **futura** (las observaciones posteriores al tiempo $t$) para estimar correctamente la probabilidad de que el sistema haya estado en un determinado estado en ese tiempo.

### Ejemplo:

Supongamos que tenemos una observación inesperada al final de la secuencia que difícilmente pudo haber ocurrido en algunos estados anteriores. Sin el paso **Backward**, el algoritmo no tendría forma de ajustar las probabilidades anteriores con base en esa nueva información.

- Por ejemplo, si en el tiempo $t=1$ se considera probable estar en el estado $S_1$, pero en el tiempo $t=3$ se observa un evento muy improbable desde $S_1$, el paso backward puede ajustar hacia abajo la probabilidad de $S_1$ en $t=1$.

Esto se debe a que los **HMM son modelos generativos de toda la secuencia**, y no solo del presente. Sin el paso backward, solo se tendría una visión parcial (solo hacia adelante) y se perdería contexto importante que afecta la inferencia.


## Task 2

In [7]:
import random

class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        sequence = []
        current_state = random.choices(self.states, weights=[self.initial_prob[s] for s in self.states])[0]
        for _ in range(length):
            obs = random.choices(self.observations, weights=[self.emission_prob[current_state][o] for o in self.observations])[0]
            sequence.append(obs)
            current_state = random.choices(self.states, weights=[self.transition_prob[current_state][s] for s in self.states])[0]
        return sequence

    def forward(self, observations):
        alpha = []
        for t, obs in enumerate(observations):
            alpha_t = {}
            for state in self.states:
                if t == 0:
                    alpha_t[state] = self.initial_prob[state] * self.emission_prob[state][obs]
                else:
                    alpha_t[state] = sum(
                        alpha[t-1][prev_state] * self.transition_prob[prev_state][state] for prev_state in self.states
                    ) * self.emission_prob[state][obs]
            alpha.append(alpha_t)
        return alpha

    def backward(self, observations):
        beta = [{} for _ in observations]
        T = len(observations)
        for state in self.states:
            beta[T-1][state] = 1
        for t in reversed(range(T-1)):
            for state in self.states:
                beta[t][state] = sum(
                    self.transition_prob[state][next_state] *
                    self.emission_prob[next_state][observations[t+1]] *
                    beta[t+1][next_state]
                    for next_state in self.states
                )
        return beta

    def compute_state_probabilities(self, observations):
        alpha = self.forward(observations)
        beta = self.backward(observations)
        gamma = []
        for t in range(len(observations)):
            prob_sum = sum(alpha[t][s] * beta[t][s] for s in self.states)
            gamma_t = {s: (alpha[t][s] * beta[t][s]) / prob_sum for s in self.states}
            gamma.append(gamma_t)
        return gamma


## **Implementación de la clase**

In [8]:
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Sunny', 'Rainy', 'Rainy']
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}
}
emission_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}
}

hmm = HMM(states, ['Sunny', 'Rainy'], initial_prob, transition_prob, emission_prob)

# 3. Secuencia generada (opcional)
obs_sequence = hmm.generate_sequence(5)
print("Secuencia Generada:", obs_sequence)

# 4. Probabilidades Forward
forward_probs = hmm.forward(observations)
print("Probabilidades Forward:")
print(forward_probs)

# 5. Probabilidades Backward
backward_probs = hmm.backward(observations)
print("Probabilidades Backward:")
print(backward_probs)

# 6. Probabilidades combinadas (estado)
state_probs = hmm.compute_state_probabilities(observations)
print("Probabilidades de Estados:")
print(state_probs)


Secuencia Generada: ['Sunny', 'Rainy', 'Sunny', 'Rainy', 'Rainy']
Probabilidades Forward:
[{'Sunny': 0.4, 'Rainy': 0.15}, {'Sunny': 0.30400000000000005, 'Rainy': 0.051000000000000004}, {'Sunny': 0.05272000000000002, 'Rainy': 0.06398000000000001}, {'Sunny': 0.013553600000000006, 'Rainy': 0.0342524}]
Probabilidades Backward:
[{'Sunny': 0.08956000000000001, 'Rainy': 0.07988}, {'Sunny': 0.11800000000000001, 'Rainy': 0.23399999999999999}, {'Sunny': 0.30000000000000004, 'Rainy': 0.5}, {'Sunny': 1, 'Rainy': 1}]
Probabilidades de Estados:
[{'Sunny': 0.7493620047692758, 'Rainy': 0.25063799523072416}, {'Sunny': 0.7503660628373008, 'Rainy': 0.24963393716269922}, {'Sunny': 0.33083713341421583, 'Rainy': 0.669162866585784}, {'Sunny': 0.283512529807974, 'Rainy': 0.7164874701920261}]


# 🔍 Análisis de Secuencia y Probabilidades en un Modelo Oculto de Markov (HMM)

---

## 📌 Secuencia Generada

La secuencia de observaciones generada por el modelo fue:
```python
['Sunny', 'Rainy', 'Sunny', 'Sunny', 'Rainy']
```
Esta secuencia será utilizada para el análisis de probabilidades a lo largo del tiempo.

---

## 📈 1. Probabilidades Forward (`α`)

Las probabilidades forward representan la **probabilidad conjunta de observar la secuencia hasta el tiempo $t$ y estar en un estado específico**:
```python
[ {'Sunny': 0.4, 'Rainy': 0.15}, {'Sunny': 0.304, 'Rainy': 0.051}, {'Sunny': 0.05272, 'Rainy': 0.06398}, {'Sunny': 0.0135536, 'Rainy': 0.0342524} ]
```

### 📝 Observaciones:
- En el tiempo $t=0$, hay una mayor probabilidad de estar en **Sunny** (0.4 vs. 0.15), influenciado por la alta probabilidad de emisión de 'Sunny' en el estado 'Sunny'.
- En $t=1$, la probabilidad en 'Sunny' sigue dominando (0.304), pero comienza a ceder terreno frente a 'Rainy'.
- A partir de $t=2$ y $t=3$, la probabilidad de estar en 'Rainy' aumenta, lo cual refleja el cambio de tendencia en las observaciones hacia valores de 'Rainy'.

---

## 📉 2. Probabilidades Backward (`β`)

Las probabilidades backward representan la **probabilidad de observar la secuencia restante desde $t+1$ hasta el final**, dado que el sistema está en un estado particular en el tiempo $t$:
```python
[ {'Sunny': 0.08956, 'Rainy': 0.07988}, {'Sunny': 0.118, 'Rainy': 0.234}, {'Sunny': 0.3, 'Rainy': 0.5}, {'Sunny': 1, 'Rainy': 1} ]
```

### 📝 Observaciones:
- En $t=3$ (último paso analizado), las probabilidades backward valen 1, ya que no hay eventos futuros a considerar.
- En $t=1$ y $t=2$, la probabilidad de continuar en la secuencia es más alta desde el estado 'Rainy', dado que las observaciones siguientes son mayormente 'Rainy'.

---

## 🔁 3. Probabilidades Posteriores de Estado (`γ`)

Estas probabilidades combinan la información de `α` y `β` para estimar la **probabilidad de que el sistema esté en un estado dado en el tiempo $t$, considerando toda la secuencia observada**:
```python
[ {'Sunny': 0.74936, 'Rainy': 0.25064}, {'Sunny': 0.75037, 'Rainy': 0.24963}, {'Sunny': 0.33084, 'Rainy': 0.66916}, {'Sunny': 0.28351, 'Rainy': 0.71649} ]
```


### 📝 Observaciones:
- En los primeros dos pasos ($t=0$ y $t=1$), el sistema tiene una fuerte inclinación hacia el estado 'Sunny', consistente con las observaciones iniciales.
- A partir de $t=2$, el modelo actualiza sus creencias y empieza a favorecer 'Rainy' como estado oculto, especialmente debido a la secuencia de observaciones 'Rainy' consecutivas.
- Esto refleja una propiedad clave de los HMMs: **la capacidad de reconsiderar hipótesis previas en función de nueva evidencia futura**.

---

## ✅ Conclusiones

- El modelo HMM ajusta sus estimaciones de estado oculto conforme avanza la secuencia observada.
- La combinación del algoritmo Forward y Backward permite capturar no solo la historia pasada, sino también la influencia del futuro sobre el presente.
- En este ejemplo, la transición progresiva de 'Sunny' a 'Rainy' como estado más probable muestra un comportamiento consistente con la evidencia proporcionada por la secuencia observada.


